# 00 — Orchestrator (LangGraph + MCP)
**Purpose:** Build the agent graph, manage state, emit telemetry, and respond.

This notebook contains the main LangGraph orchestrator that coordinates all customer care operations through a structured workflow of specialized nodes and conditional routing.


## Config & Parameters (TODO)

### Model Configuration
- **Temperature**: 0.3 (consistent, deterministic responses)
- **Top_p**: 0.9 (balanced creativity and focus)
- **Max_output_tokens**: 512 (concise, actionable responses)
- **Model**: GPT-4 or Claude-3 (high-quality reasoning)

### Operational Limits
- **Tool calls per turn**: ≤6 (prevent excessive API usage)
- **Graph steps**: ≤20 (prevent infinite loops)
- **Session timeout**: 30 minutes (automatic cleanup)
- **Memory limit**: 10,000 tokens (efficient context management)

### Path Configuration
- **Shared resources**: `shared/` (SOPs, FAQs, schemas)
- **Memory storage**: `memory/global/` and `memory/session/`
- **Log storage**: `logs/runs/<trace_id>/trace.jsonl`
- **Evaluation data**: `logs/eval/<date>/`

### Feature Flags
- **Dry-run write operations**: `true` (safe testing mode)
- **HITL enabled**: `true` (human oversight for high-impact actions)
- **Audit logging**: `true` (compliance and monitoring)
- **PII redaction**: `true` (privacy protection)
- **Policy enforcement**: `true` (SOP compliance)

### Security Settings
- **Injection detection**: `enabled` (prompt injection protection)
- **Auth level enforcement**: `strict` (access control)
- **Rate limiting**: `enabled` (DoS protection)
- **Input validation**: `enabled` (malformed request protection)


## State Object (TODO)

### Core State Schema
The shared state object is the single source of truth for LangGraph, containing all conversation context and workflow data.

```json
{
  "messages": [],
  "intent": null,
  "intent_confidence": 0,
  "auth_level": 0,
  "user_ref": null,
  "proposed_actions": [],
  "citations": [],
  "risk_flags": [],
  "tool_results": {},
  "needs_HITL": false,
  "memory_updates": []
}
```

### State Key Definitions

#### **messages**
- **Type**: Array of message objects
- **Purpose**: Complete conversation history
- **Format**: `[{"role": "user|assistant", "text": "...", "timestamp": "...", "metadata": {...}}]`
- **Usage**: Context for LLM, conversation continuity

#### **intent**
- **Type**: String or null
- **Purpose**: Classified customer intent
- **Values**: `{sales, upsell, refunds, troubleshoot, order, sop, unknown}`
- **Usage**: Routing decisions, specialized agent selection

#### **intent_confidence**
- **Type**: Float (0.0-1.0)
- **Purpose**: Confidence score for intent classification
- **Threshold**: ≥0.6 for automatic routing, <0.6 requires clarification
- **Usage**: Routing confidence, HITL triggers

#### **auth_level**
- **Type**: Integer (0-2)
- **Purpose**: User authentication level
- **Values**: 0=unauthenticated, 1=basic, 2=premium
- **Usage**: Access control, feature availability

#### **user_ref**
- **Type**: Object or null
- **Purpose**: Authenticated user profile
- **Content**: User ID, email, name, account details
- **Usage**: Personalization, order lookup, access control

#### **proposed_actions**
- **Type**: Array of action objects
- **Purpose**: Candidate actions from sub-agents
- **Format**: `[{"id": "...", "action": "...", "args": {...}, "rationale": "..."}]`
- **Usage**: Policy gate validation, HITL review

#### **citations**
- **Type**: Array of citation objects
- **Purpose**: SOP/FAQ sources for responses
- **Format**: `[{"source_path": "...", "heading": "...", "clause_id": "...", "hash": "..."}]`
- **Usage**: Response transparency, policy compliance

#### **risk_flags**
- **Type**: Array of risk strings
- **Purpose**: Security and compliance flags
- **Values**: `["injection_risk", "pii_detected", "policy_violation", "auth_insufficient"]`
- **Usage**: Security decisions, routing blocks

#### **tool_results**
- **Type**: Object
- **Purpose**: Latest MCP tool outputs
- **Format**: `{"tool_name": {"result": {...}, "timestamp": "...", "duration": ...}}`
- **Usage**: Sub-agent coordination, result sharing

#### **needs_HITL**
- **Type**: Boolean
- **Purpose**: Human-in-the-loop requirement flag
- **Triggers**: High-impact actions, low confidence, policy ambiguity
- **Usage**: Workflow routing, approval gates

#### **memory_updates**
- **Type**: Array of memory objects
- **Purpose**: Sanitized summaries for persistent storage
- **Format**: `[{"scope": "session|global", "summary": "...", "timestamp": "..."}]`
- **Usage**: Knowledge persistence, context continuity


## Telemetry Hooks (TODO)

### Event Emission System
All nodes emit structured events to the telemetry system for monitoring, debugging, and compliance purposes.

### Event Types and Schemas

#### **message_received**
```json
{
  "event": "message_received",
  "text": "Customer message content",
  "timestamp": "2025-01-15T10:30:00Z",
  "trace_id": "trace_2025-01-15_10-30-00_a1b2c3d4"
}
```

#### **triage_result**
```json
{
  "event": "triage_result",
  "intent": "refunds",
  "confidence": 0.85,
  "features_considered": ["keywords", "context", "history"],
  "timestamp": "2025-01-15T10:30:05Z"
}
```

#### **guardrails_scan**
```json
{
  "event": "guardrails_scan",
  "jailbreak_flags": ["ignore_instructions", "system_override"],
  "pii_flags": ["email_detected"],
  "safe": false,
  "timestamp": "2025-01-15T10:30:10Z"
}
```

#### **route_enter/exit**
```json
{
  "event": "route_enter",
  "node": "refunds_agent",
  "reason": "intent_classification_confidence_high",
  "timestamp": "2025-01-15T10:30:15Z"
}
```

#### **tool_call_start/end**
```json
{
  "event": "tool_call_start",
  "tool": "orders.lookup",
  "args_sanitized": {"order_id": "o_2001", "user_id": "u_***"},
  "timestamp": "2025-01-15T10:30:20Z"
}
```

#### **policy_gate_decision**
```json
{
  "event": "policy_gate_decision",
  "action": "orders.refund_apply",
  "clause_id": "RF-2.3",
  "decision": "approved",
  "justification": "Matches policy RF-2.3 for damaged item within 30 days",
  "timestamp": "2025-01-15T10:30:25Z"
}
```

#### **hitl_required/decision**
```json
{
  "event": "hitl_required",
  "reason": "high_value_refund_over_500",
  "options_summary": [{"id": "opt_1", "action": "full_refund", "amount": 299.00}],
  "timestamp": "2025-01-15T10:30:30Z"
}
```

#### **memory_write**
```json
{
  "event": "memory_write",
  "scope": "session",
  "summary": "Customer refund request processed successfully",
  "timestamp": "2025-01-15T10:30:35Z"
}
```

#### **response_sent**
```json
{
  "event": "response_sent",
  "citations": [{"source_path": "shared/sops/refunds_policy_v2024-07-01.md", "clause_id": "RF-2.3"}],
  "tokens_out": 156,
  "latency_ms": 2450,
  "timestamp": "2025-01-15T10:30:40Z"
}
```

### Telemetry Implementation

#### **Event Queue**
- **In-memory queue**: Real-time event collection
- **Batch processing**: Efficient event batching
- **Backpressure handling**: Queue size limits

#### **Log Persistence**
- **Path**: `logs/runs/<trace_id>/trace.jsonl`
- **Format**: JSON Lines (one event per line)
- **Rotation**: Daily log rotation with compression
- **Retention**: 90 days operational, 7 years compliance

#### **PII Sanitization**
- **Email masking**: `user***@example.com`
- **Phone masking**: `+1-555-****`
- **Address masking**: City/state level only
- **Credit card**: Never logged

#### **Performance Monitoring**
- **Latency tracking**: Per-node execution times
- **Token usage**: Input/output token counting
- **Error rates**: Failure tracking and alerting
- **Resource usage**: Memory and CPU monitoring


## LangGraph Nodes (TODO)

### Complete Node List with Responsibilities and I/O

#### **1. Triage**
- **Purpose**: Classify customer intent and determine routing
- **Input**: Latest user message, conversation history
- **Processing**: Intent classification using LLM with exemplars
- **Output**: `state.intent`, `state.intent_confidence`, `triage_result` event
- **Intent Types**: `{sales, upsell, refunds, troubleshoot, order, sop, unknown}`

#### **2. Guardrails.Pre**
- **Purpose**: Security scan before processing user input
- **Input**: Latest user message
- **Processing**: Injection detection, PII scanning, suspicious pattern analysis
- **Output**: `state.risk_flags`, `guardrails_scan` event
- **Action**: If `safe=false`, route to refusal path

#### **3. Auth**
- **Purpose**: User authentication and authorization
- **Input**: User identifier (email, phone, session token)
- **Processing**: Lookup user profile, determine auth level
- **Output**: `state.auth_level`, `state.user_ref`
- **Tools**: `auth.lookup_user`

#### **4. Router (Conditional)**
- **Purpose**: Route to appropriate specialized agent
- **Input**: `state.intent`, `state.intent_confidence`, `state.auth_level`
- **Processing**: Conditional routing based on intent and confidence
- **Output**: Route to specific agent or clarification request
- **Routes**: Sales/Upsell, Refunds, Troubleshoot, Order Status, SOP Lookup

#### **5. Sales/Upsell**
- **Purpose**: Generate sales recommendations and upsell offers
- **Input**: User profile, order history, current context
- **Processing**: Analyze customer needs, suggest relevant products
- **Output**: Append to `state.proposed_actions`
- **Tools**: User profile lookup, product catalog access

#### **6. Refunds**
- **Purpose**: Process refund requests and generate quotes
- **Input**: Order details, refund reason, customer context
- **Processing**: Retrieve SOP, identify policy clause, compute refund options
- **Output**: Append to `state.proposed_actions`, `state.citations`
- **Tools**: `orders.lookup`, `orders.refund_quote`, `kb.search`

#### **7. Troubleshoot**
- **Purpose**: Provide step-by-step troubleshooting guidance
- **Input**: Problem description, product information, customer context
- **Processing**: Analyze symptoms, retrieve troubleshooting guides
- **Output**: Diagnostic steps with confidence scores and citations
- **Tools**: `kb.search`, product documentation access

#### **8. Order Status**
- **Purpose**: Provide order and shipping status information
- **Input**: Order ID, customer authentication
- **Processing**: Lookup order details and shipping information
- **Output**: Status explanation with next steps (read-only)
- **Tools**: `orders.lookup`, `shipping.status`

#### **9. SOP Lookup**
- **Purpose**: Retrieve policy and procedure information
- **Input**: Search query, context requirements
- **Processing**: Semantic search across knowledge base
- **Output**: Relevant passages with canonical citations
- **Tools**: `kb.search`, `fs.read`

#### **10. PolicyGate**
- **Purpose**: Validate all proposed actions against company policies
- **Input**: `state.proposed_actions`, user auth level
- **Processing**: Check each action against relevant SOPs
- **Output**: `policy_gate_decision` events, set `state.needs_HITL`
- **Tools**: `policy.check`

#### **11. HITL (Optional)**
- **Purpose**: Human review and approval for high-impact actions
- **Input**: Action options with scoring rubric
- **Processing**: Present options to human reviewer via UI
- **Output**: `hitl_decision` event with approved option
- **Triggers**: Write operations, low confidence, policy conflicts

#### **12. Executor**
- **Purpose**: Execute approved and policy-compliant actions
- **Input**: Approved actions from HITL or PolicyGate
- **Processing**: Call underlying MCP tools (dry-run by default)
- **Output**: Action results, `tool_call_*` events
- **Tools**: All MCP tools based on action type

#### **13. Memory Write**
- **Purpose**: Persist sanitized conversation summaries
- **Input**: `state.memory_updates`
- **Processing**: Sanitize and store session/global memory
- **Output**: `memory_write` events
- **Storage**: Session memory (ephemeral), Global memory (persistent)

#### **14. Guardrails.Post**
- **Purpose**: Final security and compliance check before response
- **Input**: Generated response, citations, context
- **Processing**: PII redaction, citation validation, safety check
- **Output**: Sanitized response ready for customer
- **Requirements**: At least one valid citation for policy-based advice

#### **15. Logger**
- **Purpose**: Persist all telemetry events to disk
- **Input**: Event queue from all nodes
- **Processing**: Serialize events to JSON Lines format
- **Output**: `logs/runs/<trace_id>/trace.jsonl`
- **Features**: Compression, rotation, retention management


## Routing Edges (TODO)

### Main Workflow Path
```
Triage → Guardrails.Pre → Auth → Router → [Branch] → PolicyGate → (HITL?) → Executor → Memory → Guardrails.Post → Logger
```

### Conditional Routing Logic

#### **After Triage**
- **High Confidence (≥0.6)**: Proceed to Guardrails.Pre
- **Low Confidence (<0.6)**: Route to clarification request

#### **After Guardrails.Pre**
- **Safe (true)**: Proceed to Auth
- **Unsafe (false)**: Route to refusal response

#### **After Auth**
- **Authenticated**: Proceed to Router
- **Authentication Failed**: Route to auth error response

#### **Router Branching**
Based on `state.intent` and `state.auth_level`:

- **intent="sales"** → Sales/Upsell agent
- **intent="upsell"** → Sales/Upsell agent  
- **intent="refunds"** → Refunds agent
- **intent="troubleshoot"** → Troubleshoot agent
- **intent="order"** → Order Status agent
- **intent="sop"** → SOP Lookup agent
- **intent="unknown"** → Clarification request

#### **After PolicyGate**
- **All Actions Approved**: Proceed to HITL check
- **Any Action Denied**: Route to policy violation response
- **Mixed Results**: Route to partial approval workflow

#### **HITL Decision Point**
- **needs_HITL=true**: Route to HITL review
- **needs_HITL=false**: Proceed to Executor

#### **After HITL**
- **Approved**: Proceed to Executor
- **Rejected**: Route to rejection response
- **Revised**: Return to PolicyGate for re-validation

#### **After Executor**
- **Success**: Proceed to Memory Write
- **Failure**: Route to error handling
- **Partial Success**: Route to partial success workflow

#### **After Memory Write**
- **Success**: Proceed to Guardrails.Post
- **Failure**: Proceed to Guardrails.Post (memory optional)

#### **After Guardrails.Post**
- **Valid Response**: Proceed to Logger
- **Invalid Response**: Route to response error handling

### Checkpointing Strategy

#### **Checkpoint Locations**
- **After Router**: Save routing decision and intent classification
- **After PolicyGate**: Save policy decisions and approved actions
- **After HITL**: Save human decisions and reasoning

#### **Checkpoint Data**
```json
{
  "checkpoint_id": "checkpoint_2025-01-15_10-30-00_abc123",
  "state_snapshot": {...},
  "routing_history": [...],
  "policy_decisions": [...],
  "timestamp": "2025-01-15T10:30:00Z"
}
```

#### **Replay Capabilities**
- **Full Replay**: Reconstruct entire conversation from checkpoints
- **Partial Replay**: Resume from any checkpoint
- **Debug Mode**: Step-through execution with checkpoint inspection

### Error Handling and Recovery

#### **Node Failure Recovery**
- **Retry Logic**: Automatic retry for transient failures
- **Fallback Paths**: Alternative routing for failed nodes
- **Circuit Breakers**: Prevent cascade failures
- **Graceful Degradation**: Continue with reduced functionality

#### **State Consistency**
- **Atomic Updates**: All-or-nothing state modifications
- **Rollback Capability**: Revert to previous checkpoint on failure
- **Conflict Resolution**: Handle concurrent state modifications
- **Validation**: Ensure state integrity at each checkpoint


## Chat Loop (TODO)

### Main Conversation Flow

#### **1. Message Reception**
```python
# Receive user turn → emit message_received event
user_message = receive_customer_message()
emit_event("message_received", {"text": user_message})
```

#### **2. Graph Execution**
```python
# Step graph until response_sent event
while not conversation_complete:
    next_node = determine_next_node(state)
    state = execute_node(next_node, state)
    emit_telemetry_events()
```

#### **3. Response Generation**
```python
# Return customer reply + citations
response = generate_customer_response(state)
citations = format_citations(state.citations)
return {"response": response, "citations": citations}
```

### Conversation State Management

#### **Session Initialization**
- **Trace ID Generation**: Unique identifier for conversation
- **State Initialization**: Empty state object with default values
- **Memory Loading**: Load relevant session/global memory
- **Context Setup**: Initialize conversation context

#### **Message Processing Pipeline**
1. **Input Validation**: Sanitize and validate user input
2. **State Update**: Add user message to conversation history
3. **Graph Execution**: Run LangGraph workflow
4. **Response Generation**: Create customer-facing response
5. **Memory Update**: Persist conversation insights
6. **Telemetry**: Log all events for monitoring

#### **Conversation Termination**
- **Natural End**: Customer indicates conversation complete
- **Timeout**: 30-minute inactivity timeout
- **Error State**: Critical error requiring human intervention
- **Escalation**: Complex issue requiring human agent

### Response Handling

#### **Response Types**
- **Informational**: Order status, FAQ answers, policy explanations
- **Actionable**: Refund processing, account updates, order modifications
- **Escalation**: Complex issues requiring human intervention
- **Error**: System errors, authentication failures, policy violations

#### **Citation Requirements**
- **Policy-based advice**: Must include SOP clause citations
- **Product information**: Include product documentation references
- **Procedural steps**: Include relevant procedure references
- **General information**: Optional citations for transparency

#### **Response Formatting**
```json
{
  "message": "Customer-facing response text",
  "citations": [
    {
      "source": "Refunds Policy §2.3 (v2024-07-01)",
      "url": "internal://sops/refunds_policy_v2024-07-01.md#rf-2-3"
    }
  ],
  "actions_taken": ["refund_quoted", "policy_validated"],
  "next_steps": ["customer_confirmation", "refund_processing"],
  "trace_id": "trace_2025-01-15_10-30-00_a1b2c3d4"
}
```

### Error Handling and Recovery

#### **Error Types**
- **Input Errors**: Malformed user input, invalid requests
- **Authentication Errors**: Failed login, insufficient permissions
- **Policy Violations**: Actions not allowed by company policy
- **System Errors**: Tool failures, network issues, LLM errors
- **Business Logic Errors**: Invalid operations, data inconsistencies

#### **Error Response Strategy**
- **Graceful Degradation**: Continue with reduced functionality
- **Clear Error Messages**: User-friendly error explanations
- **Alternative Options**: Suggest valid alternatives when possible
- **Escalation Path**: Direct to human agent when needed

#### **Recovery Mechanisms**
- **Retry Logic**: Automatic retry for transient failures
- **Fallback Responses**: Default responses for common errors
- **Human Handoff**: Seamless transition to human agents
- **State Recovery**: Restore from checkpoints on failure

### Performance Optimization

#### **Response Time Targets**
- **Simple Queries**: < 5 seconds (order status, FAQ)
- **Complex Operations**: < 30 seconds (refund processing, troubleshooting)
- **Human Handoff**: < 10 seconds (escalation scenarios)
- **Error Responses**: < 3 seconds (validation failures)

#### **Caching Strategy**
- **Policy Cache**: Cache frequently accessed SOP clauses
- **User Profile Cache**: Cache authenticated user information
- **Product Cache**: Cache product information and compatibility
- **Response Cache**: Cache common responses for similar queries

#### **Resource Management**
- **Memory Limits**: Enforce conversation memory limits
- **Token Limits**: Monitor and limit LLM token usage
- **Tool Call Limits**: Prevent excessive API calls
- **Session Limits**: Manage concurrent conversation sessions
